In [ ]:
import os 
import matplotlib.pyplot as plt
import tensorflow as tf

os.listdir('/kaggle/input/labeled-chest-xray-images/chest_xray')

In [ ]:
base_dir = "/kaggle/input/labeled-chest-xray-images/chest_xray"

train_dir = f"{base_dir}/train"
val_dir   = f"{base_dir}/val"
test_dir  = f"{base_dir}/test"

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define image size and batch size
image_size = 224
batch_size = 8

# Initialize ImageDataGenerators
train_datagen = ImageDataGenerator(
    rescale=1./255,             # Normalize pixel values to [0, 1]
    rotation_range=20,         # Random rotation
    width_shift_range=0.2,     # Random horizontal shift
    height_shift_range=0.2,    # Random vertical shift
    shear_range=0.2,           # Random shear
    zoom_range=0.2,            # Random zoom
    horizontal_flip=True,      # Random horizontal flip
    fill_mode='nearest',       # Fill mode for newly created pixels
    validation_split=0.2
)

test_datagen = ImageDataGenerator(rescale=1./255)  # Only rescaling for test data


In [ ]:

# Load training and testing data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode='binary',  # Use 'categorical' if there are more than two classes
    shuffle=True
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode='binary',  # Use 'categorical' if there are more than two classes
    shuffle=False
)
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation',  # <-- use the validation part
    shuffle=False
)

In [ ]:
# class names
class_names = ['NORMAL', 'PNEUMONIA']

# Display 9 images from the first batch
images, labels = next(train_generator)  # get first batch
plt.figure(figsize=(10, 10))
for i in range(6):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i])
    plt.title(class_names[int(labels[i])])
    plt.axis("off")
plt.show()

In [ ]:
# Count images in each class (from folders)
normal_count = len(os.listdir(os.path.join(train_dir, 'NORMAL')))
abnormal_count = len(os.listdir(os.path.join(train_dir, 'PNEUMONIA')))

# Bar chart
labels_bar = ['Normal', 'Abnormal']
counts = [normal_count, abnormal_count]

plt.figure(figsize=(6, 4))
bars = plt.bar(labels_bar, counts, color=['skyblue', 'salmon'])
plt.title("Training Data Distribution")
plt.xlabel("Classes")
plt.ylabel("Number of Images")
plt.bar_label(bars)
plt.show()

In [ ]:
# Pie chart
plt.figure(figsize=(6, 4))
plt.pie(counts, labels=labels_bar, autopct='%1.1f%%', colors=['skyblue', 'salmon'])
plt.title("Training Data Distribution")
plt.show()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)


## CNN

In [ ]:
cnn_model=tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters =32,kernel_size=(3,3),input_shape=(224,224,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Conv2D(filters=16,kernel_size=(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32,activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')
    
])

In [ ]:
cnn_model.compile(#optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001)
                  optimizer='adam'
                  ,loss='binary_crossentropy',
                  metrics=['accuracy'])

In [ ]:
cnn_history=cnn_model.fit(train_generator,epochs=20, validation_data=validation_generator,batch_size=32,callbacks=[early_stop])

## Vgg19
### transfer learning

In [ ]:
from tensorflow.keras.applications import VGG19
base_vgg=VGG19(
    include_top=False,
    weights='imagenet',
    pooling='avg'
)
base_vgg.trainable=False

vgg_model = tf.keras.models.Sequential([
    base_vgg,
    tf.keras.layers.Dense(1, activation="sigmoid")  # adjust for your task
])



In [ ]:
vgg_model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])


In [ ]:
vgg_history=vgg_model.fit(train_generator,epochs=10, validation_data=validation_generator,batch_size=32,callbacks=[early_stop])


### fine tuning 

In [ ]:


# Unfreeze some of the deeper convolutional layers
base_vgg.trainable = True

# Freeze the first 15 layers to keep early features stable
for layer in base_vgg.layers[:15]: #tells TensorFlow not to update the weights of those layers during training(first 15 layers)
    layer.trainable = False

vgg_model.compile(
    optimizer='adam',  # much smaller LR
    loss='binary_crossentropy',
    metrics=['accuracy']
)

fine_tune_vgg = vgg_model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10,
    
   
)



## DenseNet121

In [ ]:
from tensorflow.keras.applications import DenseNet121

In [ ]:
base_dense=DenseNet121(
    include_top=False,
    pooling='avg',
    
)
base_dense.trainable=False
dense_model=tf.keras.Sequential([
    base_dense,
    tf.keras.layers.Dense(1, activation="sigmoid")
])

In [ ]:
dense_model.compile(
    optimizer='adam',  
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [ ]:
dense_history=dense_model.fit(train_generator,epochs=20,validation_data=validation_generator,callbacks=[early_stop])

In [ ]:
dense_model.evaluate(test_generator)

In [ ]:
final_train_acc = dense_history.history['accuracy'][-1]
final_val_acc = dense_history.history['val_accuracy'][-1]

print(f"Final Training Accuracy: {final_train_acc:.4f}")
print(f"Final Validation Accuracy: {final_val_acc:.4f}")


In [ ]:

vgg_model.save('/kaggle/working/vgg_model.h5')
cnn_model.save('/kaggle/working/cnn_model.h5')
dense_model.save('/kaggle/working/dense_model.h5')

In [ ]:
# saves history of cnn
import pickle

with open('/kaggle/working/vgg_history.pkl', 'wb') as f:
    pickle.dump(cnn_history.history, f)
